In [1]:
import pandas as pd
import numpy as np

In [2]:
feat_max = pd.read_csv('/project/DSone/as3ek/data/seem_train_2048_features_max.csv')
feat_mean = pd.read_csv('/project/DSone/as3ek/data/seem_train_2048_features.csv')

In [3]:
rna_seq = pd.read_csv('/project/DSone/as3ek/data/ee_trascript.csv')

In [9]:
rna_seq = rna_seq.drop(['Unnamed: 0'], axis=1)

In [23]:
ee_feat_max = feat_max[feat_max['fname'].str.contains('SEEM')].reset_index(drop=True)

In [37]:
ee_feat_max['fname'] = ee_feat_max['fname'].str.split('__').str[0].str.split('_').str[0] + '_' + ee_feat_max['fname'].str.split('__').str[0].str.split('_').str[1]

In [ ]:
ee_feat_max_grpd = ee_feat_max.groupby('fname', as_index=False).mean()

In [15]:
rna_seq

,A1BG,A1CF,A2M,A4GALT,A4GNT,AAAS,AACS,AADAC,AAED1,AAGAB,...,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3,FileNames
0,2.938328,19.990509,83.735050,2.656216,3.404470,17.141710,11.889671,57.604313,2.052005,22.352093,...,7.167301,6.760921,23.934557,1.351300,5.770310,2.624760,32.301320,16.912931,10.427124,SEEM_1
1,4.698266,15.687117,92.621060,1.957990,0.306087,12.404878,9.844436,84.911210,1.641775,21.663334,...,7.521921,6.561717,18.160603,1.151140,6.536171,3.283770,34.461420,14.909936,13.230409,SEEM_3
2,3.455630,13.785192,107.816505,3.281972,0.010000,13.406025,8.643127,49.870205,2.645186,18.025269,...,4.720486,4.125326,13.924426,0.878572,4.932373,2.385005,29.004587,15.764729,6.827882,SEEM_4
3,2.466141,36.609490,135.728450,3.121654,0.293090,17.032896,12.018597,91.219810,3.087415,23.395113,...,8.026385,7.405385,14.101945,2.055490,7.218950,4.769273,30.454320,25.666950,14.545485,SEEM_5
4,3.249304,30.674992,115.543710,2.711289,1.881430,19.385916,11.410261,96.130104,1.817835,18.924387,...,6.723240,6.889716,18.950125,1.843850,7.449580,3.398000,22.919546,26.630081,12.240758,SEEM_6
5,2.074514,18.763176,105.074005,2.575287,1.479770,13.688148,9.032931,93.625900,2.833032,25.686758,...,8.172430,7.193019,27.086506,1.711140,4.888920,3.276200,31.529670,12.002652,10.682680,SEEM_7
6,2.719801,15.950573,68.034310,2.213923,2.285980,17.075415,11.521301,100.788730,1.780263,30.617378,...,8.636470,11.556961,34.549603,1.866020,5.004883,3.004470,33.498417,15.114459,13.274719,SEEM_8
7,2.279317,18.339360,86.282840,2.456626,0.010000,14.365479,6.815216,78.982025,2.797190,17.103210,...,4.866239,3.538407,12.156732,1.152900,5.268969,2.530060,33.750546,18.662529,5.955720,SEEM_11
8,2.107752,31.955494,114.312450,2.481700,0.010000,14.807080,10.450998,107.538270,1.782753,18.430100,...,5.402922,4.798664,13.416936,1.471470,5.119132,3.266117,36.147793,20.236704,8.531974,SEEM_15
9,1.890398,29.776218,166.736080,3.769107,0.010000,12.085667,8.429674,114.080440,1.573020,23.783787,...,5.090937,4.893142,18.946280,1.238550,4.098496,1.981600,32.363780,17.078785,7.438259,SEEM_16


In [ ]:
pd.merge(rna_seq, ee_feat_max, left_on='FileNames', right_on='fname').corr()